In [65]:
import os
import requests
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Project ~

In [43]:
df_all = pd.read_csv('JTI 2021 October Prod logs.csv')

In [44]:
df_all.columns

Index(['Passport', 'Endpoint', 'Tenant', 'Client', 'ConfigurationMode', 'Date',
       'Details', 'IP'],
      dtype='object')

In [45]:
df_all['Tenant'].value_counts()

jti.9r1.prod.bqm       59436
JTI.9r1.dev.bqm        11344
jti.9r1.sit3.bqm        7597
jti.9r1.test.bqm        6079
jti.9r1.preprod.bqm     1436
jti.training-01         1205
jti.9r1.pentest.bqm      346
jti.9r1.preview.bqm      344
jti.preview-01           198
Name: Tenant, dtype: int64

In [46]:
df_prod = df_all.loc[df_all['Tenant'] == 'jti.9r1.prod.bqm']

In [47]:
# check how many times developers acccess application
df_prod.loc[df_all['Passport'].str.contains('@beqom.com')].count()

Passport             125
Endpoint             125
Tenant               125
Client               125
ConfigurationMode    125
Date                 125
Details              125
IP                   125
dtype: int64

In [48]:
# remove developers from statistics by creating new df
# df_prod.drop(df_prod.loc[df_prod['Passport'].str.contains('@beqom.com')].index, inplace=True)
df_prod_1 = df_prod[~df_prod['Passport'].str.contains('@beqom.com')].reset_index(drop=True)

In [49]:
# df_prod_1.loc[df_prod_1['Passport'].str.contains('@beqom.com')].count()
df_prod_1.head()

,Passport,Endpoint,Tenant,Client,ConfigurationMode,Date,Details,IP
0,WELLOS.MBOZI@JTI.COM,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:38:59,EndpointURL: https://9r1-prod-neu-app.beqom.co...,41.75.127.1
1,VIKTORIYA.VLASENKO@JTI.COM,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:08:51,EndpointURL: https://9r1-prod-neu-app.beqom.co...,178.133.65.93
2,VIKTORIYA.VLASENKO@JTI.COM,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:07:39,EndpointURL: https://9r1-prod-neu-app.beqom.co...,178.133.65.93
3,OLEKSANDR.KACHKOVSKYI@JTI.COM,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:07:35,EndpointURL: https://9r1-prod-neu-app.beqom.co...,193.109.249.240
4,OLEKSANDR.KACHKOVSKYI@JTI.COM,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:06:36,EndpointURL: https://9r1-prod-neu-app.beqom.co...,193.109.249.240


In [50]:
df_prod_1.isnull().sum()

Passport             0
Endpoint             0
Tenant               0
Client               0
ConfigurationMode    0
Date                 0
Details              0
IP                   0
dtype: int64

In [51]:
df_prod_1.dtypes

Passport             object
Endpoint             object
Tenant               object
Client               object
ConfigurationMode    object
Date                 object
Details              object
IP                   object
dtype: object

In [52]:
#lower sting in passpot column
df_prod_1['Passport'] = df_prod_1['Passport'].str.lower()
#convert Date column to date format
df_prod_1['Date'] = pd.to_datetime(df_prod_1['Date'], infer_datetime_format=True)
# df_prod = df_prod.applymap(lambda s: s.lower() if type(s) == str else s)
# df_prod.isnull().sum()

In [53]:
# df_prod_1[df_prod_1['Passport'] == 'krzysztof.krzywosz@jti.com']


In [54]:
df_prod_1.dtypes

Passport                     object
Endpoint                     object
Tenant                       object
Client                       object
ConfigurationMode            object
Date                 datetime64[ns]
Details                      object
IP                           object
dtype: object

In [59]:
# ip_list = df_beqom['IP'].to_numpy()
# print(f'Number of all IP addresses {len(ip_list)}')
# unique_ip = np.unique(ip_list)
# print(f'Number of unique IP addresses {len(unique_ip)}')

print(f"Total number of IP addresses {df_prod_1['IP'].count()}")

unique_ip = df_prod_1['IP'].unique().tolist()
print(f"Total number of UNIQUE IP addresses {len(unique_ip)}")

Total number of IP addresses 59311
Total number of UNIQUE IP addresses 15143


In [58]:
df_prod_1.isnull().sum()

Passport             0
Endpoint             0
Tenant               0
Client               0
ConfigurationMode    0
Date                 0
Details              0
IP                   0
dtype: int64

In [60]:
# Based on unique IP address get specific location
# To get specific location we use API point http://ip-api.com/batch
# When building request you need to take into consideration
# - A request needs to formatted as a JSON array, containing up to 100 IP addresses or objects.
# - This endpoint is limited to 15 requests per minute from an IP address. --> 60s/15 = 4 times per minute

In [61]:
# split list unique_ip to lists containing 100 elements only
number_elements = 100
unique_ip_split = [ unique_ip[ip_element:ip_element + number_elements] for ip_element in range(0, len(unique_ip), number_elements)]

In [62]:
len(unique_ip_split)

152

In [66]:
url = "http://ip-api.com/batch"
response_dict = []
counter = 0
total_start_time = time.time()
total_end_time = time.time()

for item in range(0, len(unique_ip_split)):
    time_start = time.perf_counter()
    params = unique_ip_split[item]
    response = requests.post(url=url, json=params).json()
#     print(len(response))
    for element in response:
        response_dict.append(element)
    time_end = time.perf_counter()
    if counter in range(1, 200, 10):
        print(f'Time to get one set of data from ip-api round {counter} total time {total_end_time - total_start_time:0.4f} seconds')
#         print(f'Time to get one set of data from ip-api {time_end - time_start:0.4f} seconds')
    time.sleep(5)
    total_start_time += time_start
    total_end_time  += time_end
    counter += 1

Time to get one set of data from ip-api round 1 total time 0.1597 seconds
Time to get one set of data from ip-api round 11 total time 2.5300 seconds
Time to get one set of data from ip-api round 21 total time 4.1240 seconds
Time to get one set of data from ip-api round 31 total time 6.3633 seconds
Time to get one set of data from ip-api round 41 total time 8.7321 seconds
Time to get one set of data from ip-api round 51 total time 11.1662 seconds
Time to get one set of data from ip-api round 61 total time 13.8499 seconds
Time to get one set of data from ip-api round 71 total time 15.6747 seconds
Time to get one set of data from ip-api round 81 total time 21.4984 seconds
Time to get one set of data from ip-api round 91 total time 23.9232 seconds
Time to get one set of data from ip-api round 101 total time 26.3753 seconds
Time to get one set of data from ip-api round 111 total time 28.5279 seconds
Time to get one set of data from ip-api round 121 total time 30.5608 seconds
Time to get one

In [67]:
len(response_dict)

15143

In [69]:
df_country_ip = pd.DataFrame(response_dict)

In [71]:
df_country_ip.to_csv('df_country_ip.csv')

In [70]:
# check if we have any faild IP recogntion
df_country_ip.loc[df_country_ip['status'] != 'success'].count()

status         0
country        0
countryCode    0
region         0
regionName     0
city           0
zip            0
lat            0
lon            0
timezone       0
isp            0
org            0
as             0
query          0
dtype: int64

In [72]:
df_prod_2 = df_prod_1.merge(df_country_ip, left_on='IP',right_on='query', how='left').drop(columns='query')

In [73]:
df_prod_2.isnull().sum()

Passport             0
Endpoint             0
Tenant               0
Client               0
ConfigurationMode    0
Date                 0
Details              0
IP                   0
status               0
country              0
countryCode          0
region               0
regionName           0
city                 0
zip                  0
lat                  0
lon                  0
timezone             0
isp                  0
org                  0
as                   0
dtype: int64

In [74]:
df_prod_2.head()

,Passport,Endpoint,Tenant,Client,ConfigurationMode,Date,Details,IP,status,country,...,region,regionName,city,zip,lat,lon,timezone,isp,org,as
0,wellos.mbozi@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:38:59,EndpointURL: https://9r1-prod-neu-app.beqom.co...,41.75.127.1,success,Malawi,...,C,Central Region,Lilongwe,,-13.9626,33.7741,Africa/Blantyre,Inq. Digital Limited,Limbe Leaf,AS37187 Inq. Digital Limited
1,viktoriya.vlasenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:08:51,EndpointURL: https://9r1-prod-neu-app.beqom.co...,178.133.65.93,success,Ukraine,...,61,Ternopil Oblast,Ternopil,46001,49.5535,25.5947,Europe/Kiev,Vodafone UA,Vodafone LLC,AS21497 PrJSC VF UKRAINE
2,viktoriya.vlasenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:07:39,EndpointURL: https://9r1-prod-neu-app.beqom.co...,178.133.65.93,success,Ukraine,...,61,Ternopil Oblast,Ternopil,46001,49.5535,25.5947,Europe/Kiev,Vodafone UA,Vodafone LLC,AS21497 PrJSC VF UKRAINE
3,oleksandr.kachkovskyi@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:07:35,EndpointURL: https://9r1-prod-neu-app.beqom.co...,193.109.249.240,success,Ukraine,...,53,Poltava Oblast,Kremenchug,39630,49.0602,33.3967,Europe/Kiev,Satellite Ltd,Satellite Ltd,AS21310 Satellite Ltd
4,oleksandr.kachkovskyi@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 12:06:36,EndpointURL: https://9r1-prod-neu-app.beqom.co...,193.109.249.240,success,Ukraine,...,53,Poltava Oblast,Kremenchug,39630,49.0602,33.3967,Europe/Kiev,Satellite Ltd,Satellite Ltd,AS21310 Satellite Ltd


In [75]:
df_prod_2.count()

Passport             59311
Endpoint             59311
Tenant               59311
Client               59311
ConfigurationMode    59311
Date                 59311
Details              59311
IP                   59311
status               59311
country              59311
countryCode          59311
region               59311
regionName           59311
city                 59311
zip                  59311
lat                  59311
lon                  59311
timezone             59311
isp                  59311
org                  59311
as                   59311
dtype: int64

In [76]:
unique_emails = df_prod_2['Passport'].unique()

In [27]:
len(unique_emails)

9708

In [77]:
unique_emails = pd.DataFrame(df_prod_2['Passport'].unique())
unique_emails.to_csv('unique_emails.csv', sep=',', index=False)

In [78]:
df_users = pd.read_csv('../NYCDSA_Project_1/bq_employees_04112021.csv')

In [85]:
df_users_1 = df_users.loc[df_users['id_rank'] == 1]
df_users_1.head()

,id_user,code_payee,PersonNr,active,firstname_user,lastname_user,login_user,Company,EmployeeGroup,EmployeeSubGroup,PersonnelArea,PersonnelSubArea,JobTitle,id,id_rank,StartDate,EndDate
0,3,59915,105542.0,Yes,Adam,Miron,adam.miron@jti.com,1431,1,7,PL24,9.0,PROCESS EXPERT MANAGER,1631873.0,1,11-10-21,31-12-99
7,4,65548,99660.0,Yes,Iqbal,Satrio Nindito,IQBAL.SATRIONINDITO@JTI.COM,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1,11-10-21,31-12-99
16,5,94954,94954.0,Yes,Krzysztof,Krzywosz,krzysztof.krzywosz@jti.com,2180,A,0P,PL01,9.0,BTS PEOPLE & CULTURE CONSULTANT,1620595.0,1,01-09-21,31-12-99
18,6,8442,8442.0,No,Maria,Labrador,maria.labrador@jti.com,1799,1,7,CH02,15.0,SEVERANCE CORPORATE COMPENSATION MANAGER,1498622.0,1,01-04-21,31-12-99
20,7,27026,58290.0,Yes,Nevena,Nikolic,nevena.nikolic@jti.com,1799,1,7,CH02,15.0,PEOPLE & CULTURE AGILE PARTNER MANAGER,1322967.0,1,01-10-20,31-12-99


In [104]:
df_users_1.isnull().sum()

id_user                0
code_payee             0
PersonNr               4
active                 0
firstname_user         0
lastname_user          3
login_user             0
Company                6
EmployeeGroup          6
EmployeeSubGroup       6
PersonnelArea          6
PersonnelSubArea       6
JobTitle            3759
id                     6
id_rank                0
StartDate              6
EndDate                6
dtype: int64

In [122]:
df_users_1.set_index('login_user').stack()

login_user                               
adam.miron@jti.com         id_user                              3
                           code_payee                       59915
                           PersonNr                      105542.0
                           active                             Yes
                           firstname_user                    Adam
                                                     ...         
maumita.chowdhury@jti.com  JobTitle          PLM STAFF BANGLADESH
                           id                           1651488.0
                           id_rank                              1
                           StartDate                   2021-03-11
                           EndDate            1999-12-31 00:00:00
Length: 573818, dtype: object

In [93]:
df_users_1.dtypes

id_user               int64
code_payee            int64
PersonNr            float64
active               object
firstname_user       object
lastname_user        object
login_user           object
Company              object
EmployeeGroup        object
EmployeeSubGroup     object
PersonnelArea        object
PersonnelSubArea    float64
JobTitle             object
id                  float64
id_rank               int64
StartDate            object
EndDate              object
dtype: object

In [98]:
df_users_1['StartDate'] = pd.to_datetime(df_users_1['StartDate']).dt.date
# df_users_1['EndDate'] = pd.to_datetime(df_users_1['EndDate'])

/var/folders/mq/vpk1f6fx6tq6_vvlc4r509n40000gn/T/ipykernel_12242/1071152501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_1['StartDate'] = pd.to_datetime(df_users_1['StartDate']).dt.date


In [86]:
df_users_1['login_user'] = df_users_1['login_user'].str.lower()

/var/folders/mq/vpk1f6fx6tq6_vvlc4r509n40000gn/T/ipykernel_12242/3379579542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_1['login_user'] = df_users_1['login_user'].str.lower()


In [103]:
df_users_1.isnull().sum()

id_user                0
code_payee             0
PersonNr               4
active                 0
firstname_user         0
lastname_user          3
login_user             0
Company                6
EmployeeGroup          6
EmployeeSubGroup       6
PersonnelArea          6
PersonnelSubArea       6
JobTitle            3759
id                     6
id_rank                0
StartDate              6
EndDate                6
dtype: int64

In [84]:
df_users_1.count()

id_user             36102
code_payee          36102
PersonNr            36098
active              36102
firstname_user      36102
lastname_user       36099
login_user          36102
Company             36096
EmployeeGroup       36096
EmployeeSubGroup    36096
PersonnelArea       36096
PersonnelSubArea    36096
JobTitle            32343
id                  36096
id_rank             36102
StartDate           36096
EndDate             36096
dtype: int64

In [88]:
df_users_1.head()

,id_user,code_payee,PersonNr,active,firstname_user,lastname_user,login_user,Company,EmployeeGroup,EmployeeSubGroup,PersonnelArea,PersonnelSubArea,JobTitle,id,id_rank,StartDate,EndDate
0,3,59915,105542.0,Yes,Adam,Miron,adam.miron@jti.com,1431,1,7,PL24,9.0,PROCESS EXPERT MANAGER,1631873.0,1,11-10-21,31-12-99
7,4,65548,99660.0,Yes,Iqbal,Satrio Nindito,iqbal.satrionindito@jti.com,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1,11-10-21,31-12-99
16,5,94954,94954.0,Yes,Krzysztof,Krzywosz,krzysztof.krzywosz@jti.com,2180,A,0P,PL01,9.0,BTS PEOPLE & CULTURE CONSULTANT,1620595.0,1,01-09-21,31-12-99
18,6,8442,8442.0,No,Maria,Labrador,maria.labrador@jti.com,1799,1,7,CH02,15.0,SEVERANCE CORPORATE COMPENSATION MANAGER,1498622.0,1,01-04-21,31-12-99
20,7,27026,58290.0,Yes,Nevena,Nikolic,nevena.nikolic@jti.com,1799,1,7,CH02,15.0,PEOPLE & CULTURE AGILE PARTNER MANAGER,1322967.0,1,01-10-20,31-12-99


In [89]:
df_prod_3 = df_prod_2.merge(df_users_1, left_on='Passport',right_on='login_user', how='left').drop(columns='login_user')

In [90]:
df_prod_3[df_prod_3['Passport'] == 'iqbal.satrionindito@jti.com']

,Passport,Endpoint,Tenant,Client,ConfigurationMode,Date,Details,IP,status,country,...,Company,EmployeeGroup,EmployeeSubGroup,PersonnelArea,PersonnelSubArea,JobTitle,id,id_rank,StartDate,EndDate
122,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-29 13:21:59,EndpointURL: https://9r1-prod-neu-app.beqom.co...,213.134.183.178,success,Poland,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
165,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-29 06:47:09,EndpointURL: https://9r1-prod-neu-app.beqom.co...,213.134.183.178,success,Poland,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
196,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-28 14:17:39,EndpointURL: https://9r1-prod-neu-app.beqom.co...,213.134.183.178,success,Poland,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
203,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-28 11:01:58,EndpointURL: https://9r1-prod-neu-app.beqom.co...,213.134.183.178,success,Poland,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
523,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-22 12:10:22,EndpointURL: https://9r1-prod-neu-app.beqom.co...,213.134.183.178,success,Poland,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59274,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2020-04-21 07:56:55,EndpointURL: https://9r1-prod-neu-app.beqom.co...,13.80.158.6,success,Netherlands,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
59283,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2020-04-20 07:48:37,EndpointURL: https://9r1-prod-neu-app.beqom.co...,13.80.158.6,success,Netherlands,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
59298,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2020-04-17 15:28:37,EndpointURL: https://9r1-prod-neu-app.beqom.co...,13.80.158.6,success,Netherlands,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99
59309,iqbal.satrionindito@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2020-04-17 09:08:10,EndpointURL: https://9r1-prod-neu-app.beqom.co...,13.80.158.6,success,Netherlands,...,1431,1,7,PL24,15.0,GLOBAL COMPENSATION MANAGER,1632919.0,1.0,11-10-21,31-12-99


In [91]:
df_prod_3.count()

Passport             59322
Endpoint             59322
Tenant               59322
Client               59322
ConfigurationMode    59322
Date                 59322
Details              59322
IP                   59322
status               59322
country              59322
countryCode          59322
region               59322
regionName           59322
city                 59322
zip                  59322
lat                  59322
lon                  59322
timezone             59322
isp                  59322
org                  59322
as                   59322
id_user              58347
code_payee           58347
PersonNr             58333
active               58347
firstname_user       58347
lastname_user        58347
Company              58333
EmployeeGroup        58333
EmployeeSubGroup     58333
PersonnelArea        58333
PersonnelSubArea     58333
JobTitle             54644
id                   58333
id_rank              58347
StartDate            58333
EndDate              58333
d

In [41]:
df_prod_3a = df_prod_3[df_prod_3['Pers.No'].isna()]

KeyError: 'Pers.No'

In [185]:
# df_prod_3a['Passport'].unique()
unique_nonemployees = pd.DataFrame(df_prod_3a['Passport'].unique())
unique_nonemployees.to_csv('file2.csv', sep=',', index=False)

In [186]:
df_prod_3a

,Passport,Endpoint,Tenant,Client,ConfigurationMode,Date,Details,IP,status,country,...,EmployeeGroup,EmployeeSubGroup,PersonnelArea,PersonnelSubArea,OrgUnit,CostCenter,active,StartDate,EndDate,dest_rank
16,ekaterina.mironenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 08:49:45,EndpointURL: https://9r1-prod-neu-app.beqom.co...,2.207.106.132,success,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,ekaterina.mironenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-11-02 08:49:43,EndpointURL: https://9r1-prod-neu-app.beqom.co...,2.207.106.132,success,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,ekaterina.mironenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-21 13:09:09,EndpointURL: https://9r1-prod-neu-app.beqom.co...,188.104.250.20,success,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,ekaterina.mironenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-21 13:05:18,EndpointURL: https://9r1-prod-neu-app.beqom.co...,188.104.250.20,success,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,ekaterina.mironenko@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-10-21 13:05:17,EndpointURL: https://9r1-prod-neu-app.beqom.co...,188.104.250.20,success,Germany,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63605,yildirim.orcun@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-01-04 11:24:23,EndpointURL: https://9r1-prod-neu-app.beqom.co...,176.233.48.10,success,Turkey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63606,yildirim.orcun@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-01-04 11:24:20,EndpointURL: https://9r1-prod-neu-app.beqom.co...,176.233.48.10,success,Turkey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63607,yildirim.orcun@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-01-04 11:24:05,EndpointURL: https://9r1-prod-neu-app.beqom.co...,176.233.48.10,success,Turkey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63700,kirstiegraceallen.carvajal@jti.com,JTI 9r1 Prod,jti.9r1.prod.bqm,JTI,ExternalIdP(JTI (Prod - SAML)),2021-01-04 04:56:21,EndpointURL: https://9r1-prod-neu-app.beqom.co...,112.201.163.233,success,Philippines,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
unique_nonemployees

,0
0,ekaterina.mironenko@jti.com
1,alaa.salah@jti.com
2,agata.gawron-prochazka@jti.com
3,jitka.kerbachova@jti.com
4,jose.pabon@jti.com
...,...
89,laura.ulak@jti.com
90,anuratha.chyrypathy@jti.com
91,yuliia.mikusheva@jti.com
92,sylwia.kilon@jti.com


In [191]:
df_prod_3.columns

Index(['Passport', 'Endpoint', 'Tenant', 'Client', 'ConfigurationMode', 'Date',
       'Details', 'IP', 'status', 'country', 'countryCode', 'region',
       'regionName', 'city', 'zip', 'lat', 'lon', 'timezone', 'isp', 'org',
       'as', 'firstname_user', 'lastname_user', 'id_user', 'PersonNr',
       'code_payee', 'JobTitle', 'Company', 'EmployeeGroup',
       'EmployeeSubGroup', 'PersonnelArea', 'PersonnelSubArea', 'OrgUnit',
       'CostCenter', 'active', 'StartDate', 'EndDate', 'dest_rank'],
      dtype='object')

In [194]:
df_prod_3['countryCode']

0        MW
1        UA
2        UA
3        UA
4        UA
         ..
64688    NL
64689    NL
64690    NL
64691    NL
64692    NL
Name: countryCode, Length: 64693, dtype: object